In [ ]:
!pip install gradio

#**Key Features**#
##Let's go through some of the key features of Gradio. This guide is intended to be a high-level overview of various things that you should be aware of as you build your demo. Where appropriate, we link to more detailed guides on specific topics.

##*Components*
##*Queuing*
##*Streaming outputs*
##*Streaming inputs*
##*Alert modals*
##*Styling*
##*Progress bars*
##*Batch functions*

#**Components**#
##Gradio includes more than 30 pre-built components (as well as many user-built custom components) that can be used as inputs or outputs in your demo with a single line of code. These components correspond to common data types in machine learning and data science, e.g. the gr.Image component is designed to handle input or output images, the gr.Label component displays classification labels and probabilities, the gr.Plot component displays various kinds of plots, and so on.

##Each component includes various constructor attributes that control the properties of the component. For example, you can control the number of lines in a gr.Textbox using the lines argument (which takes a positive integer) in its constructor. Or you can control the way that a user can provide an image in the gr.Image component using the sources parameter (which takes a list like ["webcam", "upload"]).

##**Static and Interactive Components**##

###Every component has a static version that is designed to display data, and most components also have an interactive version designed to let users input or modify the data. Typically, you don't need to think about this distinction, because when you build a Gradio demo, Gradio automatically figures out whether the component should be static or interactive based on whether it is being used as an input or output. However, you can set this manually using the interactive argument that every component supports.

##**Preprocessing and Postprocessing**##

###When a component is used as an input, Gradio automatically handles the preprocessing needed to convert the data from a type sent by the user's browser (such as an uploaded image) to a form that can be accepted by your function (such as a numpy array).

###Similarly, when a component is used as an output, Gradio automatically handles the postprocessing needed to convert the data from what is returned by your function (such as a list of image paths) to a form that can be displayed in the user's browser (a gallery of images).

###Consider an example demo with three input components (gr.Textbox, gr.Number, and gr.Image) and two outputs (gr.Number and gr.Gallery) that serve as a UI for your image-to-image generation model. Below is a diagram of what our preprocessing will send to the model and what our postprocessing will require from it.
##Images are converted to NumPy arrays because they are a common format for machine learning workflows. You can control the preprocessing using the component's parameters when constructing the component. For example, if you instantiate the Image component with the following parameters, it will preprocess the image to the PIL format instead:

In [ ]:
import gradio as gr

img = gr.Image(type="pil")

#**Queuing**#
##Every Gradio app comes with a built-in queuing system that can scale to thousands of concurrent users. You can configure the queue by using queue() method which is supported by the gr.Interface, gr.Blocks, and gr.ChatInterface classes.

##For example, you can control the number of requests processed at a single time by setting the default_concurrency_limit parameter of queue(), e.g.

In [ ]:
demo = gr.Interface(...).queue(default_concurrency_limit=5)
demo.launch()

##This limits the number of requests processed for this event listener at a single time to 5. By default, the default_concurrency_limit is actually set to 1, which means that when many users are using your app, only a single user's request will be processed at a time. This is because many machine learning functions consume a significant amount of memory and so it is only suitable to have a single user using the demo at a time. However, you can change this parameter in your demo easily.

#**Streaming outputs**#
##In some cases, you may want to stream a sequence of outputs rather than show a single output at once. For example, you might have an image generation model and you want to show the image that is generated at each step, leading up to the final image. Or you might have a chatbot which streams its response one token at a time instead of returning it all at once.

##In such cases, you can supply a generator function into Gradio instead of a regular function. Creating generators in Python is very simple: instead of a single return value, a function should yield a series of values instead. Usually the yield statement is put in some kind of loop. Here's an example of an generator that simply counts up to a given number:

In [ ]:
def my_generator(x):
    for i in range(x):
        yield 1

##You supply a generator into Gradio the same way as you would a regular function. For example, here's a a (fake) image generation model that generates noise for several steps before outputting an image using the gr.Interface class:

In [ ]:
import gradio as gr
import numpy as np
import time

# This function simulates the diffusion process
def fake_diffusion(steps):
    # Create a random number generator
    rng = np.random.default_rng()

     # Loop for the specified number of steps
    for i in range(steps):
         # Introduce a delay of 1 second to simulate processing time
         time.sleep(1)

         # Generate a random image with dimensions 600x600x3 (RGB)
         image = rng.random(size=(600, 600, 3))

        # Yield the generated image to the interface
         yield image

    # After the loop, create a final image with dimensions 1000x1000x3

    image = np.ones((1000, 1000, 3), np.uint8)
    image[:] = [255, 124, 0]  # Set the color of the final image

# Create a Gradio interface
demo = gr.Interface(
    fake_diffusion,  # The function to be called
    inputs=gr.Slider(1, 10, 3, step=1),  # Input slider with range 1-10, default value 3
    outputs="image"  # Output type is an image
)

# Launch the interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8a383293f7f4e98062.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


#**Alert modals**#
##You may wish to raise alerts to the user. To do so, raise a gr.Error("custom message") to display an error message. You can also issue gr.Warning("message") and gr.Info("message") by having them as standalone lines in your function, which will immediately display modals while continuing the execution of your function. Queueing needs to be enabled for this to work.

##Note below how the gr.Error has to be raised, while the gr.Warning and gr.Info are single lines.

In [ ]:
def start_process(name):
    gr.Info("Starting process")
    if name is None:
        gr.Warning("Name is empty")

    if success == False:
        raise gr.Error("Process failed")

#**Styling**#
##Gradio themes are the easiest way to customize the look and feel of your app. You can choose from a variety of themes, or create your own. To do so, pass the theme= kwarg to the Interface constructor. Gradio comes with a set of prebuilt themes which you can load from gr.themes.*. You can extend these themes or create your own themes from scratch - see the theming guide for more details.

##For additional styling ability, you can pass any CSS (as well as custom JavaScript) to your Gradio application. This is discussed in more detail in our custom JS and CSS guide.

In [ ]:
demo = gr.Interface(..., theme=gr.themes.Monochrome())

#**Progress bars*#
##Gradio supports the ability to create custom Progress Bars so that you have customizability and control over the progress update that you show to the user. In order to enable this, simply add an argument to your method that has a default value of a gr.Progress instance. Then you can update the progress levels by calling this instance directly with a float between 0 and 1, or using the tqdm()

In [ ]:
import gradio as gr
import time

# This function reverses a given word slowly
def slowly_reverse(word, progress=gr.Progress()):
     # Update the progress bar with a description
     progress(0, desc="Starting")

     # Simulate some processing time
     time.sleep(1)

     # Update the progress bar

     progress(0.05)

     # Initialize an empty string to store the reversed word
     new_string = ""

     # Iterate over each letter in the word
     for letter in progress.tqdm(word, desc="Reversing"):
         # Simulate processing time for each letter
         time.sleep(0.25)

         # Prepend the current letter to the new_string
         new_string = letter + new_string

     # Return the reversed word
     return new_string

# Create a Gradio interface
demo = gr.Interface(
    slowly_reverse,  # The function to be called
    gr.Text(),  # Input is a text field
    gr.Text()  # Output is a text field
)

# Launch the interface
demo.launch()

#If you use the tqdm library, you can even report progress
#updates automatically from any tqdm.tqdm that already exists
#within your function by setting the default argument
#as [gr.Progress(track_tqdm=True)]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d8c94dc75f2232714f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


#**Batch functions**#
##Gradio supports the ability to pass batch functions. Batch functions are just functions which take in a list of inputs and return a list of predictions.

##For example, here is a batched function that takes in two lists of inputs (a list of words and a list of ints), and returns a list of trimmed words as output:

In [ ]:
import time

def trim_words(words, lens):
    trimmed_words = []
    time.sleep(5)

    for w, l in zip(words, lens):
        trimmed_words.append(w[:int(l)])
    return [trimmed_words]

##The advantage of using batched functions is that if you enable queuing, the Gradio server can automatically batch incoming requests and process them in parallel, potentially speeding up your demo. Here's what the Gradio code looks like (notice the batch=True and max_batch_size=16)

In [ ]:
demo = gr.Interface(
    fn = trim_words,
    inputs=["textbox", "number"],
    outputs=gr.Textbox(label="Trimmed Words"),
    batch=True,
    max_batch_size=16

)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://95e32505f5f1bc15bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##With the gr.Blocks class:

In [18]:
import gradio as gr

# Create a Gradio interface
with gr.Blocks() as demo:
      # Create a row with input fields
      with gr.Row():
        word = gr.Textbox(label="word")  # Input field for the word
        leng = gr.Number(label="leng")  # Input field for the desired length
        output = gr.Textbox(label="Output")  # Output field for the trimmed word
      # Create another row with a button
      with gr.Row():
        run = gr.Button()  # Button to trigger the trimming function

      # Define the function to be called when the button is clicked
      event = run.click(trim_words, [word, leng], output, batch=True, max_batch_size=16)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://84fd287451d7050766.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
